### This kernel so far is almost copy from the below Kernels so of you want upvote those Kernels.
### I am just trying to learn my first ML project 
#### Update 1: Today I have learnt a little bit about ensembling and stacking . Tried to apply those concepts in this data set . It didnt do anything better now , but helped me understand the concepts a bit with a working code .

#### I referred to some existing solution for this , however I forgot which kernel it was , anyone has idea , they can point out and i will add due credit . 

https://www.kaggle.com/artkulak/ieee-fraud-simple-baseline-0-9383-lb

Nanashi : https://www.kaggle.com/jesucristo/fraud-complete-eda

Xhulu's kernel: https://www.kaggle.com/xhlulu/ieee-fraud-xgboost-with-gpu-fit-in-40s

In [ ]:
import os
import gc
gc.enable()

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import catboost
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.gridspec as gridspec
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn import metrics

from sklearn.svm import SVC
import time

In [ ]:
## Some switches 
useXGB = False
useLGBM = True
useEnsemble = False
## Accidental Switch Setting
if (useXGB) and(useLGBM):
    useEnsemble = False

In [ ]:
%%time
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

In [ ]:
## Quick Trick to Create Email Feature
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails)
    test[c + '_bin'] = test[c].map(emails)
    
    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])
    
    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    gc.collect()

In [ ]:
## Trick to create Hr Feature :
## https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
def make_day_feature(df, offset=0, tname='TransactionDT'):
    """
    Creates a day of the week feature, encoded as 0-6. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    offset : float (default=0)
        offset (in days) to shift the start/end of a day.
    tname : str
        Name of the time column in df.
    """
    # found a good offset is 0.58
    days = df[tname] / (3600*24)        
    encoded_days = np.floor(days-1+offset) % 7
    return encoded_days

def make_hour_feature(df, tname='TransactionDT'):
    """
    Creates an hour of the day feature, encoded as 0-23. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    tname : str
        Name of the time column in df.
    """
    hours = df[tname] / (3600)        
    encoded_hours = np.floor(hours) % 24
    return encoded_hours



In [ ]:
## Create Feature in Train and Test
train['hours'] = make_hour_feature(train)
test['hours']= make_hour_feature(test)

In [ ]:
## Label Encoding
for f in train.columns:
    if train[f].dtype=='object' or train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))   

In [ ]:
y_train = train['isFraud'].copy()
del train_transaction, train_identity, test_transaction, test_identity

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

del train, test

In [ ]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

#### Bucket the columns for easier use in future

In [ ]:
columns = []
for f in X_train.columns:
    columns.append(f)
    
dtcol = [columns[0]]
cardcol = columns[3:9]
ccol = columns[15:29]
dcol = columns[29:44]
mcol = columns[44:53]
vcol = columns[53:392]
idcol = columns[392:430]
customcol = columns[432:437] 


#### The below code for value density comparison of train and test are taken from Chris Deottes kernel for Malware detection

In [ ]:
# COMPARE VALUE DENSITIES FROM TWO DIFFERENT DATAFRAMES
#
# PARAMETERS
# df1: pandas.DataFrame containing variable
# df2: pandas.DataFrame containing variable
# col: column to compare between df1 and df2
# override: set to False to prevent display when variables similar
# verbose: display text summary
# scale: zooms y-axis
# title: plot title
# lab1: legend label for df1
# lab2: legend label for df2
# prefix: pre text for verbose summary
#
def comparePlot(df1, df2, col, factor=4, override=True, verbose=True, scale=0.5, title='',
                lab1='', lab2='', prefix=''):
    cv1 = pd.DataFrame(df1[col].value_counts(normalize=True).reset_index().rename({col:'train'},axis=1))
    cv2 = pd.DataFrame(df2[col].value_counts(normalize=True).reset_index().rename({col:'test'},axis=1))
    cv3 = pd.merge(cv1,cv2,on='index',how='outer')
    cv3['train'].fillna(0,inplace=True)
    cv3['test'].fillna(0,inplace=True)
    cv3 = cv3.iloc[np.lexsort((cv3['test'], -cv3['train']))]
    cv3['total'] = cv3['train']+cv3['test']
    cv3['trainMX'] = cv3['train']*factor
    cv3['trainMN'] = cv3['train']/factor
    cv3 = cv3[cv3['total']>0.0001]
    if (len(cv3)<5): return
    cv3.reset_index(inplace=True)
    MX = (cv3['test'] > cv3['trainMX'])
    mxSum = round(100*cv3.loc[MX,'test'].sum(),1)
    MN = (cv3['test'] < cv3['trainMN'])
    mnSum = round(100*cv3.loc[MN,'test'].sum(),1)
    #if override | (MX.sum()+MN.sum()>0):
    if override | (mxSum + mnSum > 1):
        plt.figure(figsize=(12,3))
        if lab1=='': lab1='Train'
        if lab2=='': lab2='Test'
        plt.plot(cv3.index,cv3['train'],linewidth=3,alpha=0.7,color='b',label=lab1)
        plt.plot(cv3.index,cv3['trainMX'],linewidth=2,alpha=1.0,linestyle=':',color='b',label=str())
        plt.plot(cv3.index,cv3['trainMN'],linewidth=2,alpha=1.0,linestyle=':',color='b',label=str())
        #plt.bar(cv3.index,cv3['test'],linewidth=3,alpha=0.7,color='g', label='Test.csv')
        plt.plot(cv3.index,cv3['test'],linewidth=3,alpha=0.7,color='g',label=lab2)
        plt.legend()
        if title=='': plt.title(col)
        else: plt.title(col+' - '+title)
        plt.xlabel(col+' values (ordered by train frequency and relabeled)')
        plt.ylabel('Frequency')
        mx = max(cv3['train'].max(),cv3['test'].max())
        #plt.ylim(0,mx*1.05)
        plt.ylim(0,mx*scale)
        plt.show()
        tempMX = cv3.loc[MX.values,['index','test']].sort_values('test',ascending=False)['index']
        tempMN = cv3.loc[MN.values,['index','test']].sort_values('test',ascending=False)['index']
        if verbose:
            if MX.sum()>0:    
                print(prefix+'Test.csv',col,'has',MX.sum(),'values 4x MORE freq than Train.csv. (',mxSum,'% of data)')
            if MX.sum()>10: print('  Top 10 by test freq:',list(tempMX)[:10])
            elif MX.sum()>0: print(list(tempMX)[:10])
            if MN.sum()>0:
                print(prefix+'Test.csv',col,'has',MN.sum(),'values 4x LESS freq than Train.csv. (',mnSum,'% of data)')
            if MN.sum()>10: print('  Top 10 by test freq:',list(tempMN)[:10])
            elif MN.sum()>0: print(list(tempMN)[:10])
    return

In [ ]:
for cols in cardcol: 
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

##### Note : Drop card1 and card2 ? The frequency of data is slightly different 

In [ ]:
for cols in ccol: 
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

*#### Note: Nothing interesting here .*

In [ ]:
for cols in dcol: 
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

#### Note : Except for D3,5,7,9,13 others have larges variance between train and test with respect to value counts . Can we drop those ?

In [ ]:
for cols in vcol[0:15]:
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

*#### Does not seem to have much problem for now . Since based on the latest clarification it was created as features by DS community of the organizers , we can probably keep them . Just few check on V201 and V258 , seen in the last version that they were important features by LGBM*

In [ ]:
for cols in vcol[200:202]: 
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

In [ ]:
for cols in vcol[250:260]: 
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

#### *Does not seem to be a big problem as of now .*

*#### Why dont mcols give an output. Need to check .*

In [ ]:
for cols in idcol: 
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

#### id 7,13,18,19,21,24,26,31 seems to have problem in distribution

In [ ]:
for cols in customcol: 
        comparePlot(X_train, X_test, cols, verbose=True, title='Test vs. Train')

*#### Newly created features seem to be okay*

In [ ]:

drop_col = dtcol +mcol[0:len(mcol)]+ idcol[5:39]  + dcol[0:len(dcol)] + ccol[4:len(ccol)]
X_train.drop(drop_col,axis=1, inplace=True)
X_test.drop(drop_col, axis=1, inplace=True)


X_train.head()

In [ ]:
X_train.fillna(-1,inplace=True)
X_test.fillna(-1,inplace=True)
X_train.head()

In [ ]:
def make_amt_feature(df, tname='TransactionAmt'):
    """
    Creates an hour of the day feature, encoded as 0-23. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    tname : str
        Name of the time column in df.
    """
    logAmt = np.log(df[tname])      
    
    return logAmt

In [ ]:
## Create Feature in Train and Test
X_train['LogAmt'] = make_amt_feature(X_train)
X_test['LogAmt']= make_amt_feature(X_test)

In [ ]:
%%time
if useXGB :
    NFOLDS = 6
    kf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=123)

    y_preds_xgb = np.zeros(X_test.shape[0])
    y_oof_xgb = np.zeros(X_train.shape[0])
    score = 0
  
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        clf = xgb.XGBClassifier(
            n_estimators=500,
            max_depth=9,
            learning_rate=0.05,
            subsample=0.9,
            colsample_bytree=0.9,
            gamma = 0.2,
            alpha = 4,
            missing = -1,
            tree_method='gpu_hist'
        )
    
        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        clf.fit(X_tr, y_tr)
        y_pred_train = clf.predict_proba(X_vl)[:,1]
        y_oof_xgb[val_idx] = y_pred_train
        print("FOLD: ",fold,' AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
        score += roc_auc_score(y_vl, y_pred_train) / NFOLDS
        y_preds_xgb+= clf.predict_proba(X_test)[:,1] / NFOLDS
    
        del X_tr, X_vl, y_tr, y_vl
        gc.collect()
    
    y_preds_xgb.reshape(-1,1)   
    y_oof_xgb.reshape(-1,1)
    print("\nMEAN AUC = {}".format(score))
    print("OOF AUC = {}".format(roc_auc_score(y_train, y_oof_xgb)))

In [ ]:
%%time
if useXGB :
    NFOLDS = 5
    kf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=0)

    y_preds_xgb1 = np.zeros(X_test.shape[0])
    y_oof_xgb1 = np.zeros(X_train.shape[0])
    score = 0
  
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        clf = xgb.XGBClassifier(
            n_estimators=1500,
            max_depth=10,
            learning_rate=0.05,
            subsample=0.9,
            colsample_bytree=0.9,
            gamma = 0.2,
            alpha = 4,
            missing = -1,
            tree_method='gpu_hist'
        )
    
        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        clf.fit(X_tr, y_tr)
        y_pred_train = clf.predict_proba(X_vl)[:,1]
        y_oof_xgb1[val_idx] = y_pred_train
        print("FOLD: ",fold,' AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
        score += roc_auc_score(y_vl, y_pred_train) / NFOLDS
        y_preds_xgb1+= clf.predict_proba(X_test)[:,1] / NFOLDS
    
        del X_tr, X_vl, y_tr, y_vl
        gc.collect()
    
    y_preds_xgb1.reshape(-1,1)   
    y_oof_xgb1.reshape(-1,1)
    print("\nMEAN AUC = {}".format(score))
    print("OOF AUC = {}".format(roc_auc_score(y_train, y_oof_xgb1)))

In [ ]:
%%time
if useXGB :
    NFOLDS = 5
    kf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)

    y_preds_xgb2 = np.zeros(X_test.shape[0])
    y_oof_xgb2 = np.zeros(X_train.shape[0])
    score = 0
  
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        clf = xgb.XGBClassifier(
            n_estimators=1500,
            max_depth=10,
            learning_rate=0.05,
            subsample=0.9,
            colsample_bytree=0.9,
            gamma = 0.2,
            alpha = 4,
            missing = -1,
            tree_method='gpu_hist'
        )
    
        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        clf.fit(X_tr, y_tr)
        y_pred_train = clf.predict_proba(X_vl)[:,1]
        y_oof_xgb2[val_idx] = y_pred_train
        print("FOLD: ",fold,' AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
        score += roc_auc_score(y_vl, y_pred_train) / NFOLDS
        y_preds_xgb2+= clf.predict_proba(X_test)[:,1] / NFOLDS
    
        del X_tr, X_vl, y_tr, y_vl
        gc.collect()
    
    y_preds_xgb2.reshape(-1,1)   
    y_oof_xgb2.reshape(-1,1)
    print("\nMEAN AUC = {}".format(score))
    print("OOF AUC = {}".format(roc_auc_score(y_train, y_oof_xgb2)))

In [ ]:
param = {'num_leaves': 30,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1}

In [ ]:
max_iter = 5
gc.collect()

In [ ]:
%%time
from sklearn.model_selection import TimeSeriesSplit

if useLGBM :
    #folds = KFold(n_splits=5, shuffle=True, random_state=15)
    folds = TimeSeriesSplit(n_splits =5)
    oof = np.zeros(len(X_train))
    #categorical_columns = [c for c in cat_cols ]
    features = [c for c in X_train.columns]
    predictions = np.zeros(len(X_test))
    start = time.time()
    feature_importance_df = pd.DataFrame()
    start_time= time.time()
    score = [0 for _ in range(folds.n_splits)]

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(X_train.iloc[trn_idx][features],
                           label=y_train.iloc[trn_idx]#,
                           #categorical_feature = categorical_columns
                          )
        val_data = lgb.Dataset(X_train.iloc[val_idx][features],
                           label=y_train.iloc[val_idx]#,
                           #categorical_feature = categorical_columns
                          )

        num_round = 500
        clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=10,
                    early_stopping_rounds = 100)
    
        oof[val_idx] = clf.predict(X_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        # we perform predictions by chunks
        initial_idx = 0
        chunk_size = 100000
        current_pred = np.zeros(len(X_test))
        while initial_idx < X_test.shape[0]:
            final_idx = min(initial_idx + chunk_size, X_test.shape[0])
            idx = range(initial_idx, final_idx)
            current_pred[idx] = clf.predict(X_test.iloc[idx][features], num_iteration=clf.best_iteration)
            initial_idx = final_idx
        predictions += current_pred / min(folds.n_splits, max_iter)
   
        print("time elapsed: {:<5.2}s".format((time.time() - start_time) / 3600))
        score[fold_] = metrics.roc_auc_score(y_train.iloc[val_idx], oof[val_idx])
        if fold_ == max_iter - 1: break
        
    if (folds.n_splits == max_iter):
        print("CV score: {:<8.5f}".format(metrics.roc_auc_score(y_train, oof)))
    else:
         print("CV score: {:<8.5f}".format(sum(score) / max_iter))


In [ ]:
feature_importance_df['fold'].value_counts()

In [ ]:
df_lose_it = feature_importance_df.loc[feature_importance_df['fold'] == 5].sort_values(by="importance", ascending=True).head(100)

In [ ]:
df_lose_it=df_lose_it.loc[df_lose_it['importance'] == 0.0000]
drop_add = []
for vals in df_lose_it['feature'] :
    drop_add.append(vals)


In [ ]:
## Fold1 Feature Importance
plt.figure(figsize=(20, 10))
sns.barplot(x="importance", y="feature", data=feature_importance_df.loc[feature_importance_df['fold'] == 1].sort_values(by="importance", ascending=False).head(20))
plt.title('LightGBM Features fold 1')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
## Fold 2 Feature Importance

plt.figure(figsize=(20, 10))
sns.barplot(x="importance", y="feature", data=feature_importance_df.loc[feature_importance_df['fold'] == 2].sort_values(by="importance", ascending=False).head(20))
plt.title('LightGBM Features fold 2')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-02.png')

In [ ]:
## Fold 3 Feature importance 

plt.figure(figsize=(20, 10))
sns.barplot(x="importance", y="feature", data=feature_importance_df.loc[feature_importance_df['fold'] == 3].sort_values(by="importance", ascending=False).head(20))
plt.title('LightGBM Features fold 3')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-03.png')

In [ ]:
## Fold 4 Feature importance 

plt.figure(figsize=(20, 10))
sns.barplot(x="importance", y="feature", data=feature_importance_df.loc[feature_importance_df['fold'] == 4].sort_values(by="importance", ascending=False).head(20))
plt.title('LightGBM Features fold 4')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-04.png')

In [ ]:
## Fold 5 Feature importance 

plt.figure(figsize=(20, 10))
sns.barplot(x="importance", y="feature", data=feature_importance_df.loc[feature_importance_df['fold'] == 5].sort_values(by="importance", ascending=False).head(20))
plt.title('LightGBM Features fold 5')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-05.png')

In [ ]:
X_train.drop(drop_add,axis=1, inplace=True)
X_test.drop(drop_add, axis=1, inplace=True)

X_train.head()

In [ ]:
%%time
from sklearn.model_selection import TimeSeriesSplit

if useLGBM :
    #folds = KFold(n_splits=5, shuffle=True, random_state=15)
    folds = TimeSeriesSplit(n_splits =5)
    oof = np.zeros(len(X_train))
    #categorical_columns = [c for c in cat_cols ]
    features = [c for c in X_train.columns]
    predictions = np.zeros(len(X_test))
    start = time.time()
    feature_importance_df = pd.DataFrame()
    start_time= time.time()
    score = [0 for _ in range(folds.n_splits)]

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(X_train.iloc[trn_idx][features],
                           label=y_train.iloc[trn_idx]#,
                           #categorical_feature = categorical_columns
                          )
        val_data = lgb.Dataset(X_train.iloc[val_idx][features],
                           label=y_train.iloc[val_idx]#,
                           #categorical_feature = categorical_columns
                          )

        num_round = 10000
        clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=200,
                    early_stopping_rounds = 200)
    
        oof[val_idx] = clf.predict(X_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        # we perform predictions by chunks
        initial_idx = 0
        chunk_size = 100000
        current_pred = np.zeros(len(X_test))
        while initial_idx < X_test.shape[0]:
            final_idx = min(initial_idx + chunk_size, X_test.shape[0])
            idx = range(initial_idx, final_idx)
            current_pred[idx] = clf.predict(X_test.iloc[idx][features], num_iteration=clf.best_iteration)
            initial_idx = final_idx
        predictions += current_pred / min(folds.n_splits, max_iter)
   
        print("time elapsed: {:<5.2}s".format((time.time() - start_time) / 3600))
        score[fold_] = metrics.roc_auc_score(y_train.iloc[val_idx], oof[val_idx])
        if fold_ == max_iter - 1: break
        
    if (folds.n_splits == max_iter):
        print("CV score: {:<8.5f}".format(metrics.roc_auc_score(y_train, oof)))
    else:
         print("CV score: {:<8.5f}".format(sum(score) / max_iter))


In [ ]:
#sample_submission['isFraud'] = 0.25 * y_preds_xgb +0.25 *y_preds_xgb1 +0.25 *y_preds_xgb2 +0.25*predictions
sample_submission['isFraud'] = predictions
sample_submission.to_csv('simple_ensemble.csv')